## Colab setup

In [ ]:
# !pip install pandas numpy awkward0 uproot3_methods matplotlib
# !pip3 install torch torchvision torchaudio

# import os
# import torch
# os.environ['TORCH'] = torch.__version__
# print(torch.__version__)

# !pip install -q torch-geometric -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html

In [ ]:
import pandas as pd
import numpy as np
import awkward0 as awkward
import uproot3_methods as uproot_methods
import matplotlib.pyplot as plt

## Open HEP datasets

### Top quark jet tagging
- https://zenodo.org/record/2603256#.YKdfqSZRVH4
- graph-level classification, regression
- ~100k jets, ~100 constituents per jet


### TrackML
- https://zenodo.org/record/4730157#.YKetjy8RoWo
- edge classification


### MLPF
- https://zenodo.org/record/4559324#.YKeuDS8RoWo
- node classification and regression
- ~50k events, ~5000 particles per event

In [ ]:
!wget -nc -O test.h5 https://zenodo.org/record/2603256/files/test.h5?download=1

In [ ]:
#Read 10000 jets from top quark jet tagging
df = pd.read_hdf("test.h5", key="table", start=0, stop=10000)

In [ ]:
df.head()

In [ ]:
#based on https://github.com/hqucms/ParticleNet/blob/master/tf-keras/convert_dataset.ipynb
def _col_list(prefix, max_particles=200):
    return ['%s_%d'%(prefix,i) for i in range(max_particles)]

def get_constituents(df):
    _px = df[_col_list('PX')].values
    _py = df[_col_list('PY')].values
    _pz = df[_col_list('PZ')].values
    _e = df[_col_list('E')].values

    mask = _e>0
    n_particles = np.sum(mask, axis=1)

    px = awkward.JaggedArray.fromcounts(n_particles, _px[mask])
    py = awkward.JaggedArray.fromcounts(n_particles, _py[mask])
    pz = awkward.JaggedArray.fromcounts(n_particles, _pz[mask])
    energy = awkward.JaggedArray.fromcounts(n_particles, _e[mask])

    p4 = uproot_methods.TLorentzVectorArray.from_cartesian(px, py, pz, energy)
    jet_p4 = p4.sum()

    eta = jet_p4.eta - p4.eta
    phi = jet_p4.delta_phi(p4)
    pt = p4.pt / jet_p4.pt
    label = df['is_signal_new'].values
    
    return pt, eta, phi, label

pt, eta, phi, label = get_constituents(df)

In [ ]:
len(pt)

In [ ]:
pt.counts

In [ ]:
bins = np.linspace(0,200,50)
plt.hist(pt[label==1].counts, bins=bins, label="signal jets", histtype="step", lw=2);
plt.hist(pt[label==0].counts, bins=bins, label="background jets", histtype="step", lw=2);
plt.ylabel("Number of jets")
plt.xlabel("Number of constituents per jet")
plt.legend(frameon=False)

## Let's plot a random sample of the signal and background jet constituents.

In [ ]:
random_indices = np.random.permutation(len(eta))
plt.figure(figsize=(15,15))
for iplt in range(1,26):
    iptcl = random_indices[iplt]
    ax = plt.subplot(5,5,iplt)
    color = "blue"
    if label[iptcl] == 1:
        color = "red"
    ax.scatter(eta[iptcl], phi[iptcl], s=100*pt[iptcl], marker="o", color=color)
    plt.xlabel("$\Delta \phi$")
    plt.ylabel("$\Delta \eta$")
    plt.title("Jet {}, $N_c={}$\n$y={}$".format(iptcl, len(eta[iptcl]), label[iptcl]))
plt.tight_layout()

## Creating a PyTorch dataset

In [ ]:
import os.path as osp

import torch
from torch_geometric.data import Data, Dataset, DataLoader

from torch_geometric.nn import knn_graph

class TopTaggingDataset(Dataset):
    def __init__(self, dataframe, knn_k=4):
        super(TopTaggingDataset, self).__init__()
        
        self.knn_k = knn_k
        pt, eta, phi, label = get_constituents(dataframe)
        
        self.pt = pt
        self.eta = eta
        self.phi = phi
        self.label = label

    def len(self):
        return len(self.pt)

    def get(self, idx):
        
        pt = torch.tensor(self.pt[idx]).to(torch.float32)
        eta = torch.tensor(self.eta[idx]).to(torch.float32)
        phi = torch.tensor(self.phi[idx]).to(torch.float32)
        
        label = torch.tensor(self.label[idx]).to(torch.float32)
        
        x = torch.stack([pt, eta, phi], axis=-1)
        
        #construct knn graph from (eta, phi) coordinates
        edge_index = knn_graph(x[:, [1,2]], k=self.knn_k)
        
        data = Data(
            x = x,
            y = label,
            edge_index = edge_index
        )
        
        return data

In [ ]:
dataset = TopTaggingDataset(df, knn_k=4)

for i in range(10):
    data = dataset.get(i)
    print(data.x.shape, data.edge_index.shape, data.y)

In [ ]:
ijet = 10
data = dataset.get(ijet)
    
plt.figure(figsize=(10, 10))
plt.scatter(data.x[:, 1], data.x[:, 2], s=1000*data.x[:, 0]);
plt.xlabel("$\Delta \eta$")
plt.xlabel("$\Delta \phi$")

In [ ]:
from torch_geometric.utils import to_dense_adj
dense_adj = to_dense_adj(data.edge_index)

plt.figure(figsize=(6,5))
plt.imshow(dense_adj[0], interpolation="none", cmap="Blues")
plt.colorbar()
plt.xlabel("Node index $i$")
plt.ylabel("Node index $j$")
plt.title("Graph adjacency matrix")

In [ ]:
from torch_geometric.utils import to_networkx
import networkx as nx

In [ ]:
nxg = to_networkx(data)
pos = {i: (data.x[i, 1], data.x[i, 2]) for i in nxg.nodes}

plt.figure(figsize=(10, 10))
ax = plt.axes()
nx.draw_networkx(nxg, pos, with_labels=False, arrows=False, node_size=1000*data.x[:, 0], node_shape="o", ax=ax)
ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)


## Batching the data

In [ ]:
loader = DataLoader(dataset, batch_size=10, shuffle=True)

ibatch = 0
for data_batched in loader:
    print(ibatch, data_batched.x.shape, data_batched.y)
    ibatch += 1
    if ibatch>5:
        break

In [ ]:
dense_adj = to_dense_adj(data_batched.edge_index)

plt.figure(figsize=(9,8))
plt.imshow(dense_adj[0], interpolation="none", cmap="Blues")
plt.colorbar()
plt.xlabel("Node index $i$")
plt.xlabel("Node index $j$")
plt.title("Graph adjacency matrix")

In [ ]:
data_batched.batch

## Training a very simple GCN

In [ ]:
from torch_geometric.nn import GCNConv, global_add_pool

class Net(torch.nn.Module):
    def __init__(self, num_node_features=3):
        super(Net, self).__init__()
        
        #(3 -> 32)
        self.conv1 = GCNConv(num_node_features, 32)
        
        #(32 -> 1)
        self.output = torch.nn.Linear(32, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        #add a batch index, in in case we are running on a single graph
        if not hasattr(data, "batch"):
            data.batch = torch.zeros(len(x), dtype=torch.int64).to(x.device)
        
        #Transform the nodes with the graph convolution
        transformed_nodes = self.conv1(x, edge_index)
        transformed_nodes = torch.nn.functional.elu(transformed_nodes)
        
        #Sum up all the node vectors in each graph according to the batch index
        per_graph_aggregation = global_add_pool(transformed_nodes, data.batch)
        
        #For each graph,
        #predict the classification output based on the total vector
        #from the previous aggregation step
        output = self.output(per_graph_aggregation)
        return torch.sigmoid(output)


In [ ]:
net = Net()

In [ ]:
net(data_batched)

In [ ]:

net.state_dict().keys()

In [ ]:
net.state_dict()["conv1.lin.weight"].shape, net.state_dict()["conv1.bias"].shape

In [ ]:
plt.title("Convolutional layer weights")
plt.imshow(net.state_dict()["conv1.lin.weight"].detach().numpy().T, cmap="Blues")
plt.xlabel("feature dimension")
plt.ylabel("output dimension")
plt.xticks([0,1,2])
plt.colorbar()

In [ ]:
plt.title("Convolutional layer bias")
plt.imshow(net.state_dict()["conv1.bias"].unsqueeze(-1).detach().numpy(), cmap="Blues")
plt.xlabel("output dimenion")
plt.ylabel("feature dimension")
plt.xticks([0])
plt.colorbar()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Net().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

model.train()
losses_train = []

for epoch in range(20):
    
    loss_train_epoch = []
    
    for data_batch in loader:
        data_batch = data_batch.to(device)
        
        optimizer.zero_grad()
        out = model(data_batch)
        loss = torch.nn.functional.binary_cross_entropy(out[:, 0], data_batch.y)
        
        loss.backward()
        loss_train_epoch.append(loss.item())
        optimizer.step()
        
    loss_train_epoch = np.mean(loss_train_epoch)
    losses_train.append(loss_train_epoch)
    print(epoch, loss_train_epoch)

In [ ]:
plt.plot(losses_train, label="training")
plt.ylabel("Loss")
plt.xlabel("epoch")

In [ ]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

In [ ]:
plt.title("Convolutional layer weights")
plt.imshow(model.state_dict()["conv1.lin.weight"].detach().cpu().numpy().T, cmap="Blues")
plt.xlabel("feature dimension")
plt.ylabel("output dimension")
plt.xticks([0,1,2])
plt.colorbar()

In [ ]:
plt.title("Convolutional layer bias")
plt.imshow(model.conv1.state_dict()["bias"].unsqueeze(-1).detach().cpu().numpy(), cmap="Blues")
plt.xlabel("feature dimension")
plt.ylabel("output dimension")
plt.xticks([0])
plt.colorbar()

In [ ]:
data = dataset.get(ijet).to(device)
embedded_nodes = model.conv1(data.x, data.edge_index)

In [ ]:
data.x.shape, embedded_nodes.shape

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(data.x.cpu().numpy(), interpolation="none", cmap="Blues")
plt.colorbar()
plt.xticks([0,1,2]);

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(embedded_nodes.detach().cpu().numpy(), interpolation="none", cmap="Blues")
plt.colorbar()

In [ ]:
model_cpu = model.to('cpu')

In [ ]:
plt.figure(figsize=(15,15))
for iplt in range(1,26):
    iptcl = random_indices[iplt]
    ax = plt.subplot(5,5,iplt)
    data = dataset.get(iptcl)
    
    pred = model_cpu(data).detach()[0,0].item()
    
    color = "blue"
    if data.y == 1:
        color = "red"
    
    nxg = to_networkx(data)
    pos = {i: (data.x[i, 1], data.x[i, 2]) for i in nxg.nodes}

    nx.draw_networkx(
        nxg, pos,
        with_labels=False,
        arrows=False,
        node_size=100*data.x[:, 0],
        node_shape="o",
        ax=ax
    )
    ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)

    plt.xlabel("$\Delta \phi$")
    plt.ylabel("$\Delta \eta$")
    plt.title("Jet {}, $N_c={}$\n$y={}$, $p={:.2f}$".format(iptcl, len(eta[iptcl]), label[iptcl], pred))
plt.tight_layout()

## Dynamic graph network

In [ ]:
class DynamicNet(torch.nn.Module):
    def __init__(self, knn_k=4, num_node_features=3):
        super(DynamicNet, self).__init__()
        
        self.knn_k = knn_k
        
        #(3 -> 32)
        self.lin1 = torch.nn.Linear(num_node_features, 32)
        
        #(32 -> 32)
        self.conv1 = GCNConv(32, 32)
            
        #(32 -> 1)
        self.output = torch.nn.Linear(32, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        #add a batch index, in in case we are running on a single graph
        if not hasattr(data, "batch"):
            data.batch = torch.zeros(len(x), dtype=torch.int64).to(x.device)
        
        x = self.lin1(x)
        
        edge_index2 = knn_graph(x, k=self.knn_k, batch=data.batch)
        x = self.conv1(x, edge_index2)
        xg = global_add_pool(x, data.batch)
        output = self.output(xg)
        
        return torch.sigmoid(output), edge_index2


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net2 = DynamicNet().to(device)

optimizer = torch.optim.Adam(net2.parameters(), lr=1e-3)

model.train()
losses_train = []

for epoch in range(20):
    
    loss_train_epoch = []
    
    for data_batch in loader:
        data_batch = data_batch.to(device)
        
        optimizer.zero_grad()
        out, _ = net2(data_batch)
        loss = torch.nn.functional.binary_cross_entropy(out[:, 0], data_batch.y)
        
        loss.backward()
        loss_train_epoch.append(loss.item())
        optimizer.step()
        
    loss_train_epoch = np.mean(loss_train_epoch)
    losses_train.append(loss_train_epoch)
    print(epoch, loss_train_epoch)

In [ ]:
net2_cpu = net2.cpu()

plt.figure(figsize=(15,15))
for iplt in range(1,26):
    iptcl = random_indices[iplt]
    ax = plt.subplot(5,5,iplt)
    data = dataset.get(iptcl)
    
    pred, edge_index = net2_cpu(data)
    pred = pred.detach().item()
    
    data.edge_index = edge_index
    
    color = "blue"
    if data.y == 1:
        color = "red"
    
    nxg = to_networkx(data)
    pos = {i: (data.x[i, 1], data.x[i, 2]) for i in nxg.nodes}

    nx.draw_networkx(
        nxg, pos,
        with_labels=False,
        arrows=False,
        node_size=100*data.x[:, 0],
        node_shape="o",
        ax=ax
    )
    ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)

    plt.xlabel("$\Delta \phi$")
    plt.ylabel("$\Delta \eta$")
    plt.title("Jet {}, $N_c={}$\n$y={}$, $p={:.2f}$".format(iptcl, len(eta[iptcl]), label[iptcl], pred))
plt.tight_layout()